# 🌲 最小生成树 (Minimum Spanning Tree, MST) - Java 笔记

## 1. 什么是最小生成树？

在一个**带权无向连通图**中，最小生成树是指一个子图，它满足以下条件：
1.  **包含所有顶点**：图中所有的 V 个顶点都在树中。
2.  **连通且无环**：是一个树结构，刚好有 V-1 条边。
3.  **权重之和最小**：所有边的权重加起来是所有可能的生成树中最小的。

**典型应用场景**：
*   **城市修路**：如何用最短的总距离连接所有城市？
*   **网络布线**：如何用最少的网线连接所有电脑？
*   **电路设计**：如何用最少的金属连接所有元件？

## 2. 核心算法

解决 MST 问题主要有两种经典算法：
1.  **Kruskal 算法 (克鲁斯卡尔)**：以**边**为核心，适合稀疏图。
2.  **Prim 算法 (普里姆)**：以**点**为核心，适合稠密图。

---

## 3. Kruskal 算法 (加边法)

### 💡 核心思想
**贪心策略**：每次都选当前图中**权重最小**的边，只要这条边不会和已有的边构成环，就把它加入到 MST 中。

### 🔧 关键技术：并查集 (Union-Find)
为了快速判断两个点是否已经连通（即是否会构成环），我们需要使用并查集。

### 📝 步骤
1.  把图中的所有边按权重**从小到大排序**。
2.  遍历排序后的边：
    *   如果这条边的两个端点当前**不在**同一个集合（不连通）：
        *   选这条边！
        *   用并查集把这两个端点所在的集合合并。
    *   如果已经在同一个集合：
        *   跳过（否则会形成环）。
3.  直到选够 V-1 条边，或者遍历完所有边。

### 💻 Java 实现

```java
import java.util.Arrays;
import java.util.Comparator;

class KruskalMST {

    // 定义边类
    static class Edge {
        int src, dest, weight;
        
        public Edge(int src, int dest, int weight) {
            this.src = src;
            this.dest = dest;
            this.weight = weight;
        }
    }

    // 并查集类 (用于检测环)
    static class UnionFind {
        int[] parent;

        public UnionFind(int n) {
            parent = new int[n];
            for (int i = 0; i < n; i++) {
                parent[i] = i; // 初始时，每个点的父节点是自己
            }
        }

        // 查找根节点 (带路径压缩)
        public int find(int x) {
            if (parent[x] != x) {
                parent[x] = find(parent[x]);
            }
            return parent[x];
        }

        // 合并两个集合
        public boolean union(int x, int y) {
            int rootX = find(x);
            int rootY = find(y);
            if (rootX != rootY) {
                parent[rootX] = rootY;
                return true; // 合并成功
            }
            return false; // 已经在同一个集合，无需合并
        }
    }

    // Kruskal 主函数
    public static int kruskal(int V, Edge[] edges) {
        // 1. 按照权重对边进行排序
        Arrays.sort(edges, Comparator.comparingInt(e -> e.weight));

        UnionFind uf = new UnionFind(V);
        int mstWeight = 0;
        int edgesCount = 0;

        // 2. 遍历每一条边
        for (Edge edge : edges) {
            // 3. 如果不构成环，就加入 MST
            if (uf.union(edge.src, edge.dest)) {
                mstWeight += edge.weight;
                edgesCount++;
                System.out.println("选边: " + edge.src + " - " + edge.dest + " 权重: " + edge.weight);
            }
        }

        return mstWeight;
    }

    public static void main(String[] args) {
        int V = 4; // 顶点数
        Edge[] edges = new Edge[] {
            new Edge(0, 1, 10),
            new Edge(0, 2, 6),
            new Edge(0, 3, 5),
            new Edge(1, 3, 15),
            new Edge(2, 3, 4)
        };

        System.out.println("最小生成树的总权重: " + kruskal(V, edges));
    }
}
```

---

## 4. Prim 算法 (加点法)

### 💡 核心思想
**贪心策略**：从一个初始点开始，像霉菌扩展一样。每次选择**连接“已访问区域”和“未访问区域”的边中权重最小的那条**，把新的点拉入伙。

### 🔧 关键技术：优先队列 (Priority Queue)
我们需要一个容器，能快速告诉我们当前所有候选边里哪条最短。小顶堆（Priority Queue）是最完美的数据结构。

### 📝 步骤
1.  任选一个点作为起点，标记为“已访问”。
2.  把这个点连接的所有边放入优先队列。
3.  当优先队列不为空时：
    *   弹出权重最小的边。
    *   检查这条边的另一头（目标点）是否已经访问过：
        *   如果**已访问**：跳过（不用处理）。
        *   如果**未访问**：
            *   将该点标记为“已访问”。
            *   累计权重。
            *   **关键**：把这个新点连接的所有邻边（且连接到未访问点的边）加入优先队列。

### 💻 Java 实现

```java
import java.util.*;

class PrimMST {

    // 邻接表节点
    static class Node {
        int dest, weight;
        public Node(int dest, int weight) {
            this.dest = dest;
            this.weight = weight;
        }
    }

    public static int prim(int V, List<List<Node>> adj) {
        // 优先队列：按权重从小到大排序 [权重, 目标点]
        // 这里存 int[] {weight, node_index}
        PriorityQueue<int[]> pq = new PriorityQueue<>(Comparator.comparingInt(a -> a[0]));

        boolean[] visited = new boolean[V];
        int mstWeight = 0;

        // 从节点 0 开始
        // 格式: {权重, 节点}
        pq.offer(new int[]{0, 0});

        while (!pq.isEmpty()) {
            int[] curr = pq.poll();
            int weight = curr[0];
            int u = curr[1];

            // 如果已经访问过，跳过
            if (visited[u]) continue;

            // 标记为已访问，并加入 MST
            visited[u] = true;
            mstWeight += weight;
            System.out.println("访问节点: " + u + " 增加权重: " + weight);

            // 遍历邻居
            for (Node neighbor : adj.get(u)) {
                if (!visited[neighbor.dest]) {
                    pq.offer(new int[]{neighbor.weight, neighbor.dest});
                }
            }
        }

        return mstWeight;
    }

    public static void main(String[] args) {
        int V = 4;
        List<List<Node>> adj = new ArrayList<>();
        for (int i = 0; i < V; i++) adj.add(new ArrayList<>());

        // 构建图 (无向图要加两次)
        // 0 - 1 (10)
        adj.get(0).add(new Node(1, 10));
        adj.get(1).add(new Node(0, 10));
        // 0 - 2 (6)
        adj.get(0).add(new Node(2, 6));
        adj.get(2).add(new Node(0, 6));
        // 0 - 3 (5)
        adj.get(0).add(new Node(3, 5));
        adj.get(3).add(new Node(0, 5));
        // 1 - 3 (15)
        adj.get(1).add(new Node(3, 15));
        adj.get(3).add(new Node(1, 15));
        // 2 - 3 (4)
        adj.get(2).add(new Node(3, 4));
        adj.get(3).add(new Node(2, 4));

        System.out.println("Prim算法最小生成树权重: " + prim(V, adj));
    }
}
```

## 5. 总结对比

| 特性 | Kruskal 算法 | Prim 算法 |
| :--- | :--- | :--- |
| **核心对象** | **边 (Edge)** | **点 (Vertex)** |
| **数据结构** | 并查集 (Union-Find) | 优先队列 (Priority Queue) |
| **时间复杂度** | $O(E \log E)$ 或 $O(E \log V)$ | $O(E \log V)$ (二叉堆) |
| **适用场景** | **稀疏图** (边少点多) | **稠密图** (边多点少) |
| **现实类比** | 拼图：先把最匹配的碎片拼起来 | 铺路：从起点开始一步步往外修 |
